# Libraries

In [1]:
import pandas as pd
import math
import numpy as np

# Generating original dataset

In [2]:
# gowalla/checkins.csv é o arquivo gowalla_geo_checkins.csv gerado pelo Paulo
g_dataset = pd.read_csv('../Dados/checkins_data/general/gowalla_geo_checkins.csv', index_col=False, usecols = ['userid', 'datetime', 'lat', 'lng', 'placeid', 'categoryid', 'NAME'])
g_dataset = g_dataset.dropna()

In [3]:
g_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 567352 entries, 0 to 875915
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   userid      567352 non-null  int64  
 1   placeid     567352 non-null  int64  
 2   datetime    567352 non-null  object 
 3   lng         567352 non-null  float64
 4   lat         567352 non-null  float64
 5   categoryid  567352 non-null  int64  
 6   NAME        567352 non-null  object 
dtypes: float64(2), int64(3), object(2)
memory usage: 34.6+ MB


In [4]:
g_dataset['country'] = 'United States'
g_dataset = g_dataset.rename(columns={'NAME': 'state'})

In [5]:
# datetime converter
g_dataset['datetime'] = pd.to_datetime(g_dataset['datetime'])
order_cols = ['userid', 'categoryid', 'placeid', 'datetime', 'lat', 'lng', 'country', 'state']
g_dataset = g_dataset[order_cols]

In [6]:
cols = ['userid', 'category', 'placeid', 'local_datetime', 'latitude', 'longitude', 'country_name', 'state_name']
# Criar um dicionário que mapeia os nomes atuais das colunas para os novos nomes
maps = dict(zip(g_dataset.columns, cols))

# Renomear as colunas usando o método rename()
g_dataset = g_dataset.rename(columns=maps)

In [7]:
# category_names.csv é o arquivo category_names.csv gerado pelo Paulo

categories = pd.read_csv('../Dados/checkins_data/general/category_names.csv')

In [8]:
g_dataset = pd.merge(g_dataset, categories, left_on="category", right_on="categoryid")
g_dataset['category'] = g_dataset['name']
g_dataset.drop(['name', 'categoryid'], axis=1, inplace=True)

In [9]:
g_dataset

,userid,category,placeid,local_datetime,latitude,longitude,country_name,state_name
0,1338,Nightlife,482954,2011-06-23 02:24:22+00:00,44.934198,-93.177506,United States,Minnesota
1,116691,Nightlife,482954,2011-06-05 17:42:29+00:00,44.934198,-93.177506,United States,Minnesota
2,59161,Nightlife,482954,2011-01-14 01:39:11+00:00,44.934198,-93.177506,United States,Minnesota
3,59161,Nightlife,482954,2010-07-23 01:32:22+00:00,44.934198,-93.177506,United States,Minnesota
4,149777,Nightlife,482954,2010-03-18 01:40:40+00:00,44.934198,-93.177506,United States,Minnesota
...,...,...,...,...,...,...,...,...
567347,57784,Outdoors,862189,2010-05-02 14:18:15+00:00,30.331140,-87.139284,United States,Florida
567348,8852,Outdoors,1006077,2011-02-20 19:56:00+00:00,48.535292,-123.014495,United States,Washington
567349,8852,Outdoors,1576881,2010-08-12 22:16:05+00:00,33.529635,-117.773644,United States,California
567350,8852,Outdoors,1576881,2010-08-11 23:16:16+00:00,33.529635,-117.773644,United States,California


In [10]:
g_dataset = g_dataset.sample(frac=0.3)
g_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 170206 entries, 140262 to 223031
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype              
---  ------          --------------   -----              
 0   userid          170206 non-null  int64              
 1   category        170206 non-null  object             
 2   placeid         170206 non-null  int64              
 3   local_datetime  170206 non-null  datetime64[ns, UTC]
 4   latitude        170206 non-null  float64            
 5   longitude       170206 non-null  float64            
 6   country_name    170206 non-null  object             
 7   state_name      170206 non-null  object             
dtypes: datetime64[ns, UTC](1), float64(2), int64(2), object(3)
memory usage: 11.7+ MB


In [109]:
g_dataset = g_dataset.sample(frac=0.3)
g_dataset.to_csv('gowalla/checkins.csv', index=False)